In [ ]:
pip install heartpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.1 MB/s eta 0:00:00


In [ ]:
annotations_train = ['gamer1-annotations.csv', 'gamer2-annotations.csv', 'gamer3-annotations.csv', 'gamer4-annotations.csv']
ppg_train = ['gamer1-ppg-2000-01-01.csv', 'gamer1-ppg-2000-01-02.csv','gamer2-ppg-2000-01-01.csv', 'gamer2-ppg-2000-01-02.csv','gamer3-ppg-2000-01-01.csv', 'gamer3-ppg-2000-01-02.csv', 'gamer4-ppg-2000-01-01.csv', 'gamer4-ppg-2000-01-02.csv']

In [ ]:
# returns dataframe of ppg signal based on filename (file from kaggle input folder)
# removes non-finite values
# moving average filter

def read_ppg(filename):
    df = pd.read_csv('/kaggle/input/5-gamers/' + filename)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df = df.fillna(0)
    df.astype({'Red_Signal': 'int64'}).dtypes
    df.Red_Signal.rolling(50, min_periods=1).mean() # moving average
    return df["Red_Signal"]

In [ ]:
# returns dataframe of selected features from the heartpy processing result removing NaN values

def pick_features(all_measures, selected):
    x_train = []
    for i in range(len(all_measures['bpm'])):
        row = []
        for cat in selected:
            value = all_measures[cat][i]
            row.append(remove_nan(value))
        x_train.append(row)
    return x_train


In [ ]:
# if value is NaN return 0 else returns value

def remove_nan(value):
    if math.isnan(value):
        return 0
    return value

In [ ]:
# filters and processes ppg dataframe with heartpy

def process_ppg(data):
    sr = 100
    filtered = hp.filter_signal(data, [0.5, 15], sample_rate=sr, order=3, filtertype='bandpass')

    working_data, measures = hp.process_segmentwise(filtered, sample_rate=sr, segment_width=40, segment_overlap=0.25, segment_min_size=30)
    return  working_data, measures

In [ ]:
 def x_data(filename, selected_features):

    data = read_ppg(filename)

    working_data, measures = process_ppg(data)

    return pick_features(measures, selected_features)

In [ ]:
def y_data(filename, size):
    data = []
    sleepiness = []
    with open('/kaggle/input/5-gamers/' + filename) as file:
        file.readline()
        for row in file:
            time, event, value = row.strip().split(',', 2)
            if event == 'Stanford Sleepiness Self-Assessment (1-7)':
                value = int(value)
                if value < 4:
                    sleepiness.append(0)
                else:
                    sleepiness.append(1)

    for i in range(len(sleepiness)):
        data = data + [sleepiness[i]] * math.ceil(size/25)

    return data[:size]

In [ ]:
def read_gamer(x_file1, x_file2, y_file, features):
    x_result = x_data(x_file1, features)
    x_result = np.concatenate((x_result, x_data(x_file2, features)), axis=0)
    y_result = y_data(y_file, len(x_result))
    return x_result, y_result

# Reading data

In [ ]:
features = ['bpm', 'ibi', 'breathingrate']

**Training data**

In [ ]:
x_train = []
y_train = []

for i in range(4):
    x, y = read_gamer(ppg_train[i*2], ppg_train[i*2+1], annotations_train[i], features)

    for line in x:
        x_train.append(line)
    y_train = y_train + y

NameError: name 'pd' is not defined

In [ ]:
# Oversample imbalanced dataset with SMOTE
counter_original = Counter(y_train)
print(counter_original)


# oversampling the dataset

oversample = SMOTE(sampling_strategy=0.35)
x_over, y_over = oversample.fit_resample(x_train, y_train)
x_train, y_train = x_over, y_over


# undersampling (gave worse results)

#undersample = RandomUnderSampler(sampling_strategy=0.5)
#x_under, y_under = undersample.fit_resample(x_train, y_train)
#x_train, y_train = x_under, y_under

counter = Counter(y_train)
print(counter)

NameError: name 'Counter' is not defined

**Test data**

In [1]:
x_test, y_test = read_gamer('gamer5-ppg-2000-01-01.csv', 'gamer5-ppg-2000-01-02.csv', 'gamer5-annotations.csv', features)

NameError: name 'read_gamer' is not defined

**Visualisation**

In [ ]:
plt.figure(1)
fig = plt.figure(figsize = (3, 4))
plt.bar(['Alert', 'Too tired'], counter_original.values(), width=0.5)
plt.title("Before")

plt.figure(2)
fig = plt.figure(figsize = (3, 4))
plt.bar(['Alert', 'Too tired'], counter.values(), width=0.5)
plt.title("After SMOTE")

In [ ]:
bpm = []
ibi = []
breath = []

for row in x_train[:2000]:
    bpm.append(row[0])
    ibi.append(row[1])
    breath.append(row[2])

plt.figure(1)
plt.scatter(ibi, bpm, c=y_train[:2000])
plt.xlabel('interbeat interval')
plt.ylabel('BPM')

plt.figure(2)
plt.scatter(breath, bpm, c=y_train[:2000])
plt.xlabel('breathing rate')
plt.ylabel('BPM')

plt.figure(3)
plt.scatter(ibi, breath, c=y_train[:2000])
plt.xlabel('interbeat interval')
plt.ylabel('breathing rate')

plt.show()

In [ ]:
# Moving average
df_ppg1_1 = pd.read_csv('/kaggle/input/5-gamers/gamer1-ppg-2000-01-01.csv', encoding='utf-8')
clean1_1 = df_ppg1_1.copy()
clean1_1['Red_Signal'] = df_ppg1_1.Red_Signal.rolling(30, min_periods=1).mean()

df_ppg1_1.iloc[200:2000].plot()
clean1_1.iloc[200:2000].plot()

In [2]:
hrdata = hp.get_data('/kaggle/input/5-gamers/gamer1-ppg-2000-01-01.csv', column_name='Red_Signal')
working_data, measures = hp.process(hrdata, sample_rate=100)
hp.plotter(working_data, measures, title='Heart Beat Detection on Noisy Signal', figsize=(12,4))

#zoomed in
hp.plotter(working_data, measures, figsize=(12,4))
plt.xlim(230, 250)

NameError: name 'hp' is not defined

# Classification algorithms

In [ ]:
algorithms = ['Decision tree', 'Random forest', 'KNN', 'Gaussian Naive Bayes', 'Multinomial Naive Bayes']

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [ ]:
dt = DecisionTreeClassifier(criterion='gini', max_depth = 3, random_state=8).fit(x_train, y_train)

In [ ]:
rf = RandomForestClassifier(n_estimators=150, max_depth=15, random_state=0).fit(x_train, y_train)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=20).fit(x_train, y_train)

In [ ]:
gnb = GaussianNB().fit(x_train, y_train)

In [ ]:
mnb = MultinomialNB().fit(x_train, y_train)

In [ ]:
models = [dt, rf, knn, gnb, mnb]

In [ ]:
accuracy_df = pd.DataFrame(columns=['model', 'accuracy', 'precision', 'recall', 'f1-score'])
results = []


for i in range(len(models)):
    prediction = models[i].predict(x_test)

    print(algorithms[i], "confusion matrix")
    print(confusion_matrix(y_test, prediction))

    recall = recall_score(y_test, prediction)
    precision = precision_score(y_test, prediction)
    acc = accuracy_score(y_test, prediction)
    f1 = f1_score(y_test, prediction)
    accuracy_df = accuracy_df.append({'model': algorithms[i], 'accuracy': acc, 'precision' : precision, 'recall' : recall, 'f1-score' : f1}, ignore_index=True)

print(accuracy_df)

In [ ]:
counter = Counter(y_test)
print(counter)

In [ ]:
plt.plot(y_test)
plt.plot(knn.predict(x_test))
#plt.plot(dt.predict(x_test))
#plt.plot(rf.predict(x_test))

NameError: name 'plt' is not defined